In [60]:
import sort_RAFT_table as srt
import pandas as pd
import numpy as np
import plotly.express as px
import re
from scipy.optimize import curve_fit

In [61]:
# ToDo: Kinetik Kurven erstellen,
#  90% oder max als max gemessen als conversion, als Beschreibung für diese nehmen,
#  in ist Monomer und Mn dann
#       sortieren nach Umsatz dahingehend dann nach kürzester Zeit
#       Tabellenausgabe dazu Kinetik Kurve, reaktionsbedingungen (LM, Raft agent, Zeit...)
#  am besten als interaktive website Tabelle.

# get all conversion values,
conversion_list = []
for column in srt.df.columns:
    if "conversion" in column:
        conversion_list.append(column)
print(conversion_list)

# get the hours from the headers with regex
hours_list = []
two_digit_regex = r"\d+"
for column in conversion_list:
    hours_list.append(int(re.findall(two_digit_regex, column)[0]))

print(hours_list)

['t0h-conversion', 't1h-conversion', 't2h-conversion', 't4h-conversion', 't8h-conversion', 't15h-conversion']
[0, 1, 2, 4, 8, 15]


In [62]:
def change_timeformat(time_format):
    h_m_s = str(time_format).split(":")
    m_format = int(h_m_s[0]) * 60 + int(h_m_s[1]) + int(h_m_s[2]) / 60
    return m_format

def change_timeformat_h(time_format):
    h_m_s = str(time_format).split(":")
    h_format = int(h_m_s[0]) + (int(h_m_s[1]) + int(h_m_s[2]) / 60) /60
    return h_format

# creating a table as a lookup to correct all sample measurement times
exact_times = pd.read_excel(srt.INPUT_FILE_PATH, sheet_name="exact sampling times")

time_correction_df = pd.DataFrame(data=exact_times.iloc[3:12, 3:])
time_correction_df.columns = exact_times.loc[2][3:]

time_correction_df.reset_index(inplace=True, drop=True)
ext_time_corr_df_data =[]
for row in time_correction_df.iterrows():
    reactor_nr = row[1]["Reactor"]
    if type(reactor_nr) == int or reactor_nr == "closing of reactors":
        row[1]["Reactor"] = str(reactor_nr)
        ext_time_corr_df_data.append(row[1])
    else:
        reactor_nr = (row[1]["Reactor"]).split("+")
        for nr in reactor_nr:
            row[1]["Reactor"] = nr
            ext_time_corr_df_data.append(row[1].copy())
ext_time_corr_df = pd.DataFrame(data=ext_time_corr_df_data, columns=exact_times.loc[2][3:])
ext_time_corr_df.reset_index(drop=True, inplace=True)
print("The timer was reset to 0 after the first closing of reactors in the \"t = 0\"-sampling-step")
ext_time_corr_df.columns = ["Reactor", 0, 1, 2, 4, 6, 8, 10, 15]

# change time format to minutes and set
time_cols = ext_time_corr_df.columns.difference(["Reactor"])
ext_time_corr_df[time_cols] = ext_time_corr_df[time_cols].apply(lambda x: [change_timeformat_h(d) for d in x])
ext_time_corr_df[0] = ext_time_corr_df[0].apply(lambda x: 0 )

# set index to reactor
ext_time_corr_df.set_index("Reactor", inplace=True)

# Sampling 6h and 10h were only used for sec not NMR -> no conversion for these.
skip_hours_mask = [True if h not in hours_list else False for h in ext_time_corr_df.columns]
ext_time_corr_df.drop(columns=time_cols[skip_hours_mask], inplace=True)

ext_time_corr_df



The timer was reset to 0 after the first closing of reactors in the "t = 0"-sampling-step


,0,1,2,4,8,15
Reactor,,,,,,
1,0,1.005556,2.011111,4.018056,8.016667,15.016667
2,0,1.005556,2.011111,4.018056,8.016667,15.016667
3,0,1.073611,2.068056,4.077778,8.090278,15.098611
4,0,1.073611,2.068056,4.077778,8.090278,15.098611
5,0,1.131944,2.125000,4.143056,8.180556,15.200000
6,0,1.131944,2.125000,4.143056,8.180556,15.200000
7,0,1.191667,2.184722,4.205556,8.256944,15.291667
8,0,1.191667,2.184722,4.205556,8.256944,15.291667
9,0,1.251389,2.241667,4.273611,8.338889,15.381944


In [63]:
# srt.df.columns
ext_time_corr_df.loc["15"]


0      0.000000
1      1.437500
2      2.430556
4      4.475000
8      8.581944
15    15.658333
Name: 15, dtype: float64

In [64]:
# plotting the kinetic curves of the RAFT polymerization
# generate a reformatted table with a entries per sample analysis taken described with a column of the right time

# create a dataframe with just the information needed to append to
kinetic_curves = []
for index, polymerisation_kinetic in srt.df.iterrows():
    # get the right hours list
    # polymerisation_kinetic["reactor"]

    kinetic_curve_entries = pd.DataFrame(
        {"conversion" : polymerisation_kinetic[conversion_list], "time" : hours_list, "reactor" : polymerisation_kinetic["reactor"]})
    kinetic_curve_entries["experiment"] = str(index)+"_"+polymerisation_kinetic["monomer"]+"_"+polymerisation_kinetic["RAFT-Agent"]+"_"+polymerisation_kinetic["solvent"]

    # the times are dependent on the current reactor, get current
    current_reactor_nr = str(polymerisation_kinetic["reactor"])
    current_time_list = ext_time_corr_df.loc[current_reactor_nr]
    kinetic_curve_entries["time"] = list(current_time_list)

    kinetic_curves.append(kinetic_curve_entries)

kinetic_curves_df = pd.concat(kinetic_curves)

In [65]:
kinetic_curves_df
# kinetic_curves[1]["experiment"].iloc[1]
# px.line(kinetic_curves_df[:20], x="time", y="conversion", color="experiment")

,conversion,time,reactor,experiment
t0h-conversion,0,0.000000,1,0_6_A_DMF
t1h-conversion,0.086971,1.005556,1,0_6_A_DMF
t2h-conversion,0.104365,2.011111,1,0_6_A_DMF
t4h-conversion,0.205776,4.018056,1,0_6_A_DMF
t8h-conversion,0.33213,8.016667,1,0_6_A_DMF
...,...,...,...,...
t1h-conversion,0.298969,1.373611,14,324_10_E_DMSO
t2h-conversion,0.436426,2.369444,14,324_10_E_DMSO
t4h-conversion,1.0,4.409722,14,324_10_E_DMSO
t8h-conversion,1.0,8.501389,14,324_10_E_DMSO


In [66]:
# and fit curve and the derivation
# find the maximum of the derivation

'''
deprecated: more theoretical accurate version but practically worse applicable
    # L is responsible for scaling the output range from [0,1] to [0,L]
    # b
    # k is responsible for scaling the input, which remains in (-inf,inf)
    # x0
def sigmoid (x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)


def sigmoid_derivative(x, L ,x0, k, b): # ToDo: check if this is the correct derivative
    y = (L*k*np.exp(-k*(x-x0)))/((np.exp(-k*(x-x0))+1)**2)
    return (y)
'''
def neg_growth(x, L, k):
    y = L * (1 - np.exp(k*(-x)))
    return y

def neg_growth_derivative(x, L, k):
    y = L * k * np.exp(k*(-x))
    return y

def linear_growth(x, m):
    y = m * x
    return y

def linear_growth_derivative(x, m):
    return m

xrange = np.arange(-0.2, 5, 0.1)
example_fig = px.line()
for testparams in ([1,1], [2,1], [1,2]):
    example_fig.add_scatter(x=xrange, y=neg_growth(xrange, *testparams))
    example_fig.add_scatter(x=xrange, y=neg_growth_derivative(xrange, *testparams), opacity=0.5, line=dict(dash="dot"))

print("visualisation of the functions")
# example_fig.show()

visualisation of the functions


In [72]:
fig = px.line(title="Kinetic Curve Fit", labels={"x":"time", "y":"conversion"})
hard_to_fit = [] # array for kinetic fits that meet a filter criteria further down the code
colors = px.colors.qualitative.Plotly # set up a simple color palette
extended_xdata = np.linspace(-1, 16.5, 100)


def add_fits_to_plot(fit_func, fit_func_derivative, fit_func_params, idx):
    marker_dict = dict(color=colors[idx%len(colors)]) # set same colors per kinetic
    fig.add_scatter(
        x=extended_xdata, y=fit_func(extended_xdata, *fit_func_params),
        opacity=1, line=dict(dash="dot"), name=f"{fit_func.__name__} fit", marker=marker_dict)
    print(fit_func_params)
    fig.add_scatter(
        x=extended_xdata, y=fit_func_derivative(extended_xdata, *fit_func_params),
        opacity=0.3, line=dict(dash="dash"), name=f"{fit_func.__name__} fit", marker=marker_dict)

for idx, kinetic_curve in enumerate(kinetic_curves):
    ydata = kinetic_curve["conversion"].values
    xdata = kinetic_curve["time"].values

    # choosing a few test function
    if not idx in (1, 2):
        continue

    # fitting section
    p0 = [max(ydata), 0.1] # this is a mandatory initial guess
    cv_data = curve_fit(f=neg_growth, xdata=xdata, ydata=ydata, p0=p0, nan_policy="omit", maxfev=800*10)
    popt, pcov = cv_data
        # fit with linear growth
    p0_l = [max(ydata)/7]
    cv_data_l = curve_fit(f=linear_growth, xdata=xdata, ydata=ydata, p0=p0_l, nan_policy="omit")
    popt_l, pcov_l = cv_data_l

    # calculate the squared error of fit and data points
    y_st =[]
    x_st = []
    for y_val, x_val in zip(ydata, xdata):
        if np.isnan(y_val) or np.isnan(x_val):
            continue
        y_st.append(y_val)
        x_st.append(x_val)
    y_st = np.array(y_st)
    x_st = np.array(x_st)
    squared_error = np.sum((y_st - neg_growth(x_st, *popt))**2)
    print("{:.6f}".format(squared_error), str(idx) + ": The optimal parameters are " + " and ".join("%.3f" % x for x in popt) + ". The variance is " + " and respectively ".join("{:.3f} {:.3f}".format(x, y) for x, y in pcov) + ".")

    # filter for inspection
    # if not ((popt[1]) < 0.001).any(): continue
    # hard_to_fit.append(idx)

# ''' comment out if no plot is needed.
    marker_dict = dict(color=colors[idx%len(colors)]) # set same colors per kinetic
    fig.add_scatter(x=xdata, y=ydata, mode="lines+markers", opacity=1, name=kinetic_curve["experiment"].iloc[1], marker=marker_dict)
    add_fits_to_plot(neg_growth, neg_growth_derivative, popt, idx)
    add_fits_to_plot(linear_growth, linear_growth_derivative, [popt_l], idx)


fig.update_layout(
    yaxis=dict(
        range=[-0.1,1]
    ),
    xaxis_title="Time [h]",
    yaxis_title="Conversion [%]"
)
fig.show()
# '''
print(hard_to_fit, len(hard_to_fit)/len(kinetic_curves))



0.000594 1: The optimal parameters are 0.802 and 0.133. The variance is 0.001 -0.000 and respectively -0.000 0.000.
[0.80249768 0.13316079]
[array([0.05294653])]
0.000175 2: The optimal parameters are 1.067 and 0.024. The variance is 0.124 -0.003 and respectively -0.003 0.000.
[1.06738558 0.0238886 ]
[array([0.0219887])]


[] 0.0


In [68]:
neg_growth(10, 1, 1)

0.9999546000702375